In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
file =('//content/drive/MyDrive/one_hot_encoded_accident_risk_dataset.csv')

In [ ]:
data = pd.read_csv(file)
data.head()

,Accident,Weather_Clear,Weather_Fog,Weather_Rain,Weather_Snow,Road_Type_City,Road_Type_Highway,Road_Type_Rural,Road_Condition_Dry,Road_Condition_Icy,Road_Condition_Snow-covered,Road_Condition_Wet,Traffic_Density_High,Traffic_Density_Low,Traffic_Density_Medium,Time_of_Day_Afternoon,Time_of_Day_Evening,Time_of_Day_Morning,Time_of_Day_Night
0,No Accident,0,0,0,1,1,0,0,0,1,0,0,0,1,0,1,0,0,0
1,Accident,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0
2,No Accident,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,1
3,No Accident,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0
4,Accident,0,0,0,1,0,0,1,0,0,0,1,1,0,0,1,0,0,0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Spliting the dataset into training and testing sets (80-20 split)
X = data.drop('Accident', axis=1)  # Features
y = data['Accident']  # Target variable

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Initialize the model
model = RandomForestClassifier(random_state=42)

In [ ]:
# Train the model
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
# Evaluate the model
y_pred = model.predict(X_test) # Predict the target variable for the test set
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7333333333333333


In [ ]:
!pip install xgboost==1.7.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 5.2 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.1.1
    Uninstalling xgboost-2.1.1:
      Successfully uninstalled xgboost-2.1.1


In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier(
    n_estimators=100,  # Number of boosting rounds (trees)
    learning_rate=0.1,  # Step size shrinkage used in update to prevents overfitting
    max_depth=3,  # Maximum depth of a tree
    objective='binary:logistic',  # Loss function for binary classification
    random_state=42  # Random seed for reproducibility
)

In [ ]:
# Convert y_train to numeric type
y_train = y_train.astype(int)

# Train the model
model.fit(X_train, y_train)

# Predict the target variable for the test set
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_test = y_test.astype(int)
print(f"Accuracy: {accuracy}")

print(classification_report(y_test, y_pred))

Accuracy: 0.7333333333333333
              precision    recall  f1-score   support

           0       0.69      0.75      0.72        12
           1       0.82      0.78      0.80        18

    accuracy                           0.77        30
   macro avg       0.76      0.76      0.76        30
weighted avg       0.77      0.77      0.77        30



In [ ]:
#X_test is your test data and model is already trained
predictions = model.predict(X_test)

# Mapping predictions to output text
for pred in predictions:
    if pred == 1:
        print("High Risk of Accident")
    else:
        print("Low Risk of Accident")

High Risk of Accident
High Risk of Accident
High Risk of Accident
Low Risk of Accident
Low Risk of Accident
Low Risk of Accident
High Risk of Accident
Low Risk of Accident
High Risk of Accident
Low Risk of Accident
High Risk of Accident
Low Risk of Accident
High Risk of Accident
Low Risk of Accident
High Risk of Accident
Low Risk of Accident
High Risk of Accident
High Risk of Accident
High Risk of Accident
High Risk of Accident
Low Risk of Accident
High Risk of Accident
High Risk of Accident
High Risk of Accident
Low Risk of Accident
Low Risk of Accident
High Risk of Accident
Low Risk of Accident
Low Risk of Accident
High Risk of Accident


In [ ]:
import pickle

# Save the trained model to a file
filename = 'trained_model.pkl'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
!fuser -k 8080/tcp

In [ ]:
from flask import Flask, request, jsonify
import pickle

app = Flask(__name__)

# Load the trained model
model = pickle.load(open('trained_model.pkl', 'rb'))

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()  # Get input data as JSON
    # Preprocess the input data if necessary (e.g., one-hot encoding)
    # Make predictions using the loaded model
    prediction = model.predict([data])[0]
    # Return the prediction as a JSON response
    return jsonify({'prediction': prediction})

from flask import Flask, request, jsonify
import pickle

app = Flask(__name__)

# Load the trained model
model = pickle.load(open('trained_model.pkl', 'rb'))

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()  # Get input data as JSON
    # Preprocess the input data if necessary (e.g., one-hot encoding)
    # Make predictions using the loaded model
    prediction = model.predict([data])[0]
    # Return the prediction as a JSON response
    return jsonify({'prediction': prediction})

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=8080)

!pip install pyngrok==4.1.1
!ngrok authtoken <your_auth_token>  # Replace with your ngrok auth token
from pyngrok import ngrok
public_url = ngrok.connect(8080).public_url
print(f"Public URL: {public_url}")

!pip install pyngrok==4.1.1
!ngrok authtoken <your_auth_token>  # Replace with your ngrok auth token
from pyngrok import ngrok
public_url = ngrok.connect(8080).public_url
print(f"Public URL: {public_url}")

 * Serving Flask app '__main__'
 * Debug mode: on


Address already in use
Port 8080 is in use by another program. Either identify and stop that program, or start the server with a different port.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/serving.py", line 757, in __init__
    self.server_bind()
  File "/usr/lib/python3.10/http/server.py", line 137, in server_bind
    socketserver.TCPServer.server_bind(self)
  File "/usr/lib/python3.10/socketserver.py", line 466, in server_bind
    self.socket.bind(self.server_address)
OSError: [Errno 98] Address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-22-2830aaf4ebf0>", line 36, in <cell line: 35>
    app.run(debug=True, host='0.0.0.0', port=8080)
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1191, in run
    run_simple(t.cast(str, host), port, self, **options)
  File "/usr/local/lib/python3.10/dist-packages/werkzeu

TypeError: object of type 'NoneType' has no len()